In [85]:
from IPython.display import clear_output
import numpy as np

Regressor class

In [349]:
class Regressor:

    def __init__(self, A_DIM, R_DIM):
        self.A_DIM = A_DIM
        self.R_DIM = R_DIM
        self.W = np.array([np.random.random(
            (self.A_DIM, self.R_DIM)), np.random.random((self.R_DIM, self.A_DIM))])
        self.label_norm = 1

    def validate(self, a, b):
        self.variance, self.max_error, self.accuracy = self.metrics = self.compare_vector(
            a, b)

    def compare_vector(self, a, b):
        c = a - b
        c = np.squeeze(c)
        v = np.var(c)
        m = np.max(c)
        a = np.exp(- v / m) * 100
        return v, m, a

    def dropout_matrix(self, percent, shape):
        x = np.random.choice(2, shape, p=[1 - percent, percent])
        return x

    def fitUtil(self, training_data, training_label, alpha=0.0001, retain=1):

        hidden_layer = (training_data * self.dropout_matrix(retain,
                        training_data.shape[1])).dot(self.W[0])
        predicted_label = (
            hidden_layer * self.dropout_matrix(retain, hidden_layer.shape[1])).dot(self.W[1])

        error = training_label - predicted_label

        dW0 = - \
            training_data.T.dot(
                2 * error.dot(self.W[1].T)) / training_data.size
        dW1 = - hidden_layer.T.dot(2 * error) / training_data.size

        self.W[0] -= dW0 * alpha
        self.W[1] -= dW1 * alpha

    # Applies cross validation
    def fit(self, vecs, vecsy, epochs, **kwargs):

        '''vecs /= np.linalg.norm(vecs)
        vecsy /= np.linalg.norm(vecs)'''

        vecs = np.array(np.split(vecs, 10))
        vecsy = np.array(np.split(vecsy, 10))

        for i in range(10):

            c_v_training_data, c_v_training_label = np.vstack(
                np.delete(vecs, i, axis=0)), np.vstack(np.delete(vecsy, i, axis=0))
            c_v_testing_data, c_v_testing_label = vecs[i], vecsy[i]

            for epoch in range(epochs // 10):
                self.fitUtil(c_v_training_data, c_v_training_label, **kwargs)

            clear_output()
            self.validate(self.predict(c_v_testing_data), c_v_testing_label)
            print("Metrics", i, ":", self.metrics)

    def predict(self, data):
        return data.dot(self.W[0]).dot(self.W[1])

    def encode(self, data):
        return data.dot(self.W[0])

    def decode(self, data):
        return data.dot(self.W[1])

    def __str__(self):
        return '\n'.join(["Error variance " + str(self.variance),
                          "Error amplitude " + str(self.max_error),
                          "Accuracy " + str(self.accuracy),
                          "Weights" + str(self.W)])

    def __repr__(self):
        return '\n'.join(["Error variance " + str(self.variance),
                          "Error amplitude " + str(self.max_error),
                          "Accuracy " + str(self.accuracy),
                          "Weights" + str(self.W)])


In [355]:
class DataGenerator:
    
    def __init__(self, dim_a, dim_b, rotation_count = 0):
        self.actual_dim = dim_a
        self.reduced_dim = dim_b
        self.r = np.identity(self.actual_dim)
        for i in range(rotation_count):
            self.r = self.r.dot(self.set_rotational_matrix())


    def generate(self, data_count = 1000, noise = 0):
        
        vecs = (np.random.random((data_count, self.reduced_dim)) * 2) - 1
        vecs = np.append(vecs, np.random.random((data_count, self.actual_dim - self.reduced_dim)) * noise, axis = 1)
        vecs = vecs.dot(self.r)
        return vecs

    def set_rotational_matrix(self):
    
        theta = 2 * np.pi * np.random.random()
        axes = np.random.choice(self.actual_dim, 2, replace=False)

        i, j = axes[0], axes[1]

        rotational_matrix = np.identity(self.actual_dim)
        rotational_matrix[i][i] = rotational_matrix[j][j] = np.cos(theta)
        rotational_matrix[i][j] = np.sin(theta)
        rotational_matrix[j][i] = -np.sin(theta)

        return rotational_matrix

In [381]:
np.set_printoptions(suppress=True)
np.set_printoptions(precision=4)
datagen = DataGenerator(7, 4, rotation_count = 242)

In [385]:

regressor = Regressor(7, 4)

vecs = datagen.generate(10000, noise=0.1)
regressor.fit(vecs, vecs, 20000, alpha = 1, retain = 1)
v1 = datagen.generate(30, noise=0)
regressor.compare_vector(regressor.predict(v1), v1)

Metrics 9 : (0.0012908356263618115, 0.08911843017390622, 98.56198987692196)


(6.219967466146553e-07, 0.0024818728658584366, 99.97494155238367)

In [386]:
val = datagen.generate(1)
e = regressor.encode(val)
val, e, regressor.decode(e)

(array([[ 0.004 , -0.3477, -0.207 ,  0.9609, -0.8392,  0.1387, -0.0139]]),
 array([[-0.5392,  0.6071, -0.0591, -0.9951]]),
 array([[ 0.0025, -0.3468, -0.2084,  0.9609, -0.8393,  0.1383, -0.0139]]))